# Readme Panel Lémanique
Ce notebook reprend les données brutes GPS 2023 du Panel Lémanique et en fourni une première phase de nettoyage et de filtrage des données. A savoir deux niveau de "perte de signal" identifiés par les colonnes 'low_quality_legs_1' (env. 5% de traces éliminées) et 'low_quality_legs_2' (env. 7% de traces éliminées).
## GPS tracking
Les données GPS sont issues de la phase de collecte sur 21 jours au printemps 2023.
## Fichiers de base utilisés
- Brut: gps_panel_lemanique_by_motion_tag.csv
- Lines: lines.geojson, fichié fourni par Elisa Tirindelli le 17/08/23 --> renommé legs.geojson
- Points: points.geojson, fichié fourni par Elisa Tirindelli le 17/08/23 --> renommé staypoints.geojson
- Fichiers de raccordement: Localisation_domicile.csv par Florian Masse (trouvé sur le serveur LASUR)
- Géoinformation: Verkehrszonen_Schweiz_NPVM_2017.shp, Zone de Trafic du modèle voyageur suisse
- Questionnaire: EPFL_vague1_v4.csv, fourni par Alexis Gumy le 21/09/23
## Nettoyage par Elisa
- élimination des déplacements qui se répètent pour chaque personne plus qu’une fois (pour l'elimiation de bias dans des données)
- élimination des déplacements “triangles” (déplacement qui revient au même endroit plusieurs fois), les déplacements qui partent (ou arrivent) deux fois du (au) même endroits
- élimination des trajets (dans un même déplacement) qui partent plus tard qu’une demi heure après le trajet avant
- corriger les déplacements qui contienne plus qu’un trajet sur la même ligne (regrouper tous les trajets consecutives sur une ligne dans un seul trajet)
- corriger le temps de trajet (aussi la distance) du déplacement (les recalculer pour tenir en compte le trajets qui ont été éliminé)
- réfléchir sur les déplacements qui comprennent plus que 3/4 trajets TP (assez improbable)
## Nettoyage complémentaires par Marc-Edouard (voir dans le notebook ci-dessous)
- Enlever les étapes avec perte de signal (i.e. discontinuous legs) -> perte de 530 traces / 669808
- Segmentation des données par Canton pour faciliter la manipulation des données
- Gérer les legs non géolocalisés (beeline between OD)
- Calcul du nombre d'observation moyen par répondant.e

## Spotted issues
- 'CH14886', 'CH15539' are duplicates in the vague1_v4 file
- 'FR13508', 'CH8035', 'CH14765' are not in the vague1_v4 file but appear in the gps file

In [ ]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from shapely import geometry, ops
from shapely.geometry import MultiLineString, LineString, Point
import os

import time

## Data loading and preparation

In [ ]:
%%time

# Define the CRS you want to use (e.g., EPSG:4326 for WGS84)
target_crs = 'EPSG:4326'

# load geo data
legs = pd.read_pickle('../Data/dumps_motiontag/storyline_formated/legs.pkl')


#### Vérifier si nous avons que des linestrings
Si non, se réferer au notebook `module0_parsestoryline`

In [ ]:
legs.geometry.geom_type.value_counts()

## Nettoyage des pertes de signal

In [ ]:
## DEFINE THE MODE SPLIT TO APPLY DIFFERENT THRESHOLDS TO EACH MODE

# We propose two level of threshold to be more or less selective in the thresholds (e.g., road_psr_1 and road_psr_2, etc)

mode_road = ['Mode::Car', 'Mode::Motorbike', 'Mode::Bus', 'Mode::Tram', 'Mode::Subway', 'Mode::KickScooter', 'Mode::LightRail', 'Mode::Other','Mode::TaxiUber', 'Mode::Carsharing', 'Mode::Ecar']
road_psa_1 = 8000
road_psr_1 = 0.8
road_psa_2 = 5000
road_psr_2 = 0.6

mode_rail = ['Mode::Train','Mode::RegionalTrain']
rail_psa_1 = 100000
rail_psr_1 = 0.65
rail_psa_2 = 85000
rail_psr_2 = 0.50

mode_active = ['Mode::Bicycle', 'Mode::Ebicycle', 'Mode::Walk']
active_psa_1 = 800
active_psr_1 = 0.8
active_psa_2 = 750
active_psr_2 = 0.7

mode_plane_boat = ['Mode::Boat', 'Mode::Airplane']
plane_boat_psa_1 = 0
plane_boat_psr_1 = 0
plane_boat_psa_2 = 0
plane_boat_psr_2 = 0

# Create dictionaries for each category
road_category_1 = {'modes': mode_road, 'psa': road_psa_1, 'psr': road_psr_1}
rail_category_1 = {'modes': mode_rail, 'psa': rail_psa_1, 'psr': rail_psr_1}
active_category_1 = {'modes': mode_active, 'psa': active_psa_1, 'psr': active_psr_1}
plane_boat_category_1 = {'modes': mode_plane_boat, 'psa': plane_boat_psa_1, 'psr': plane_boat_psr_1}

road_category_2 = {'modes': mode_road, 'psa': road_psa_2, 'psr': road_psr_2}
rail_category_2 = {'modes': mode_rail, 'psa': rail_psa_2, 'psr': rail_psr_2}
active_category_2 = {'modes': mode_active, 'psa': active_psa_2, 'psr': active_psr_2}
plane_boat_category_2 = {'modes': mode_plane_boat, 'psa': plane_boat_psa_2, 'psr': plane_boat_psr_2}

# Create a dictionary to store the categories
signal_categories = [{
    'road': road_category_1,
    'rail': rail_category_1,
    'mode_active': active_category_1,
    'mode_plane_boat': plane_boat_category_1},{
    'road': road_category_2,
    'rail': rail_category_2,
    'mode_active': active_category_2,
    'mode_plane_boat': plane_boat_category_2}]

# Example: Accessing values for the 'road' category
print("Category: road")
print("Modes:", signal_categories[1]['road']['modes'])
print("PSA:", signal_categories[1]['road']['psa'])
print("PSR:", signal_categories[1]['road']['psr'])

In [ ]:
# Function to calculate the maximum distance in meters between two points in a LineString
def calculate_max_distance(line):
    # Extract the coordinates of the LineString into a list of points
    points = list(line.coords)
    
    # Initialize a list to store the distances between consecutive points
    distances = []

    # Iterate through the points to calculate and store the distances
    for i in range(len(points) - 1):
        point1 = Point(points[i])
        point2 = Point(points[i + 1])
        distance = point1.distance(point2)
        distances.append(distance)

    # Find the maximum distance from the list of distances
    max_distance = max(distances)
    
    return max_distance

In [ ]:
%%time
## !!! This one takes a while (around 60min) !
legs_ = legs.to_crs(crs="EPSG:2056").copy()
# Apply the calculate_max_distance function to the GeoSeries and store the result in a new column
legs_['max_signlalloss_meters'] = legs_.apply(lambda row: calculate_max_distance(row['geometry']), axis=1)

# Compute the lenght of each leg
legs_['length_leg'] = legs_['geometry'].apply(lambda geom: geom.length)

# Compute the relative signal loss
legs_['rel_max_signalloss'] = legs_['max_signlalloss_meters'].div(legs_['length_leg'])

# Save as temp file in case of crash
#legs_.to_pickle("../Data/temp_files/legs_rel_max_signal_loss.pkl")

In [ ]:
%%time
##TRY this parralilazation approach instead ?
#import dask.dataframe as dd
#
#legs_ = legs.to_crs(crs="EPSG:2056").copy()
## Convert GeoPandas dataframe to Dask GeoDataFrame
#legs_dask = dd.from_pandas(legs_, npartitions=7)
#
## Define a function to calculate max distance
#def calculate_max_distance_dask(geom):
#    # Your calculate_max_distance function here
#    pass
#
## Apply the function to the Dask GeoDataFrame
#legs_dask['max_signlalloss_meters'] = legs_dask['geometry'].apply(calculate_max_distance_dask, meta=('geometry', 'float'))
#legs_dask['length_leg'] = legs_dask['geometry'].apply(lambda geom: geom.length, meta=('geometry', 'float'))
#legs_dask['rel_max_signalloss'] = legs_dask['max_signlalloss_meters'] / legs_dask['length_leg']
#
## Compute the results
#legs_ = legs_dask.compute()

In [ ]:
%%time

# Add a column to flag the legs that we want to filter out
legs_['low_quality_legs_1'] = 0
legs_['low_quality_legs_2'] = 0
# Flag the low quality legs

for k, signal_categories_ in enumerate(signal_categories):
    if k == 0:
        threshold_col = 'low_quality_legs_1'
    elif k == 1:
        threshold_col = 'low_quality_legs_2'
    print('NIVEAU DU SEUIL : ', k+1)
    print('-------------------------')
    for cat in signal_categories_:
        #if cat == 'road':
        #    continue
        #else:
        legs_.loc[(legs_['mode'].isin(signal_categories_[cat]['modes'])) & 
                 ((legs_.max_signlalloss_meters > signal_categories_[cat]['psa']) | 
                 (legs_.rel_max_signalloss > signal_categories_[cat]['psr'])), threshold_col] = 1

        lost_traces = len(legs_.loc[(legs_['mode'].isin(signal_categories_[cat]['modes'])) 
                          & (legs_[threshold_col] == 1)]) / len(legs_.loc[legs_['mode'].isin(signal_categories_[cat]['modes'])]) * 100
        
        print('Category : ', cat, 
              ' | PSA : ', signal_categories_[cat]['psa'], 
              ' | PSR : ', signal_categories_[cat]['psr'], 
              ' \n Part des traces perdues : ', round(lost_traces, 1), '%')
    print('-------------------------')

# Identify the users who are always bad in terms on signal acquisition
legs_avg_signal_loss = legs_[['user_id_motiontag', 'max_signlalloss_meters', 'rel_max_signalloss']].groupby('user_id_motiontag').mean()#.unique()
legs_avg_signal_loss.reset_index(inplace=True)
list_of_bad_users_1 = legs_avg_signal_loss.loc[(legs_avg_signal_loss.rel_max_signalloss >
                                              legs_avg_signal_loss.rel_max_signalloss.quantile(0.99)) |
                                             (legs_avg_signal_loss.max_signlalloss_meters > 
                                              legs_avg_signal_loss.max_signlalloss_meters.quantile(0.99)), 'user_id_motiontag'].unique()
list_of_bad_users_2 = legs_avg_signal_loss.loc[(legs_avg_signal_loss.rel_max_signalloss >
                                              legs_avg_signal_loss.rel_max_signalloss.quantile(0.99)) |
                                             (legs_avg_signal_loss.max_signlalloss_meters > 
                                              legs_avg_signal_loss.max_signlalloss_meters.quantile(0.99)), 'user_id_motiontag'].unique()
legs_.loc[legs_.user_id_motiontag.isin(list_of_bad_users_1), 'low_quality_legs_1'] = 1
legs_.loc[legs_.user_id_motiontag.isin(list_of_bad_users_2), 'low_quality_legs_2'] = 1
print('List of users with constant low signal quality : ', list_of_bad_users_2, 
      '\n eq. to ', round(len(list_of_bad_users_2) / len(legs_) * 100, 2), '%')
print('-------------------------')


In [ ]:
legs_['usr_w_constant_bad_signal'] = 0
legs_.loc[legs_['user_id_motiontag'].isin(list_of_bad_users_1) | legs_['user_id_motiontag'].isin(list_of_bad_users_2), 'usr_w_constant_bad_signal'] = 1


In [ ]:
#SAVE TO PICKLES
legs_.to_crs(crs=target_crs).to_pickle("../Data/dumps_motiontag/storyline_time_space_filters/legs.pkl")


### Control the loss

In [ ]:
len(legs_[legs_.low_quality_legs_1 ==1]) / len(legs_)

In [ ]:
len(legs_[legs_.low_quality_legs_2 == 1]) / len(legs_)

### Test the different thresholds (for sensitivity analyse)

In [ ]:
for cat in signal_categories[0]:
    print('\n---')
    print(cat)
    df_sorted = legs_.loc[legs_['mode'].isin(signal_categories_[cat]['modes'])].sort_values(by=["rel_max_signalloss", "max_signlalloss_meters"])
    # Calculate the number of rows to keep 98% of the data
    num_rows_to_keep = int(0.95 * len(df_sorted))
    
    # Select the rows that represent the top 98% of the data
    df_filtered = df_sorted.iloc[:num_rows_to_keep]
    
    # Get the threshold values for "rel_max_signalloss" and "max_signlalloss_meters"
    print('Threshold for max_signlalloss_meters', df_filtered["max_signlalloss_meters"].max())
    print('Threshold for rel_max_signalloss', df_filtered["rel_max_signalloss"].max())

In [ ]:
len(legs_.loc[(legs_['mode'].isin(mode_active)) & 
            ((legs_.max_signlalloss_meters > 750) | 
            (legs_.rel_max_signalloss > 0.7))]) / len(legs_.loc[legs_['mode'].isin(mode_road)])

### Re-import pickles to save it to shapefiles and recombined the split pickles (in canton)

In [ ]:
%%time
# List of pickle file names
pickle_files = [
    'legs_FRA_filtered',
    'legs_GG_FRA_filtered',
    'legs_VD_part_1_filtered',
    'legs_VD_part_2_filtered',
    'legs_VD_part_3_filtered',
    'legs_GE_filtered'
]

intput_directory = 'gps_canton'

output_directory = 'gps_canton/shp'
os.makedirs(output_directory)

# Load and concatenate the pickle files
dfs = []
for file in pickle_files:
    df = pd.read_pickle(os.path.join(intput_directory, f'{file}.pkl'))
    df.to_crs(crs=target_crs).to_file(os.path.join(output_directory, f'{file}.shp'))
    dfs.append(df)

# Concatenate the DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Dump the combined DataFrame to a pickle file
combined_df.to_pickle('gps_canton/legs_filtered.pkl')
combined_df.to_crs(crs=target_crs).to_file('gps_canton/shp/legs_filtered.shp')

### For more testing

In [ ]:
## USE THIS LINE TO SAVE EXAMPLES OF HIGH SIGNAL LOSS
#legs_.loc[legs_.max_signlalloss_meters > legs_.max_distance_meters.quantile(0.75)].to_crs(crs=target_crs).to_file('gg_fra_75_quant.geojson', driver='GeoJSON')

#legs_.loc[(legs_['mode'].isin(mode_road)) & (legs_.max_signlalloss_meters < road_psa) & (legs_.rel_max_signalloss < road_psr)].to_crs(crs=target_crs).to_file('gg_fra_mode_road_.geojson', driver='GeoJSON')

#legs_.loc[(legs_['mode'].isin(mode_active)) & (legs_.max_signlalloss_meters > active_psa) & (legs_.rel_max_signalloss > active_psr)].to_crs(crs=target_crs).to_file('gg_fra_mode_active.geojson', driver='GeoJSON')


In [ ]:
##We can access the coordinates of a Linestring as follows:
#legs.geometry[30831] reads the object linestring
#type(legs.geometry[30831]) must be a shapely.geometry.linestring.LineString
#list(legs_.geometry[30831].coords) displays the coordinate tuples from which we can compute the point-to-point distance